## **IMPORT LIBRARY**

In [2]:
import wfdb as wf
import neurokit2 as ECG
from scipy import signal as Sig
import numpy as np
from numpy.polynomial import chebyshev
import os
import warnings

# Directory Path Saving
os.chdir('..')
dir = os.getcwd()
print(dir)

/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial


In [ ]:
warnings.filterwarnings("ignore")

## **CATEGORIES HEALTHY PATIENTS**

In [ ]:
# Specifying Dataset Directory
os.chdir(dir + '/Dataset')
Patients_List = os.listdir()
os.chdir(dir)


# Healthy control List
HC = []

for patient in Patients_List:
    os.chdir(dir + '/Dataset/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')] 
    List = []
    for Audio in F:
        _, fields = wf.rdsamp( dir + '/Dataset/' + patient +'/' + Audio.replace('.dat',''))
        if 'Healthy control' in fields['comments'][4]:
            List.append(Audio.replace('.dat',''))
    if 'Healthy control' in fields['comments'][4]:
        HC.append([patient, List])


## **FUNCTION DEFINITION**

In [ ]:
def Feature_Extraction(Patient, Audio, N, SR, lead, Peak_type, file):
    ''' 
        Function for extracting Features from Beatwise Based on Interval

        Parameters:

        Patient - Name of the folder which holds the Audio file
        Audio - Name of the audio file which is used to extract features
        N - Degree at which the Chebyshev Coefficients are extracted
        SR - Sampling rate
        lead - Index of the ECG signal Lead used
        Peak_type - Interval of the Beat (R-R , P-P, Q-Q, S-S or T-T)
        file - Object of the file which used to write the Features into

        Returns:

        None
    
    '''
    try:
        signal_array, _ = wf.rdsamp(f'/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial/Dataset/{Patient}/{Audio}')
        signal = signal_array[:,lead]

        signal = ECG.ecg_clean(signal, sampling_rate = SR, method='pantompkins1985')
        

        _,R_Peaks = ECG.ecg_peaks(signal, method="pantompkins1985")
        #print('Signal Size: ' + str(signal.shape[0]) )
        _,Peaks = ECG.ecg_delineate(signal, R_Peaks, sampling_rate = SR, show_type= 'all', method='cwt')

        Peaks.update(R_Peaks)

        Selected_Peaks = Peaks[f"ECG_{Peak_type}_Peaks"]
        #print(Selected_Peaks)

        for i in range(1, len(Selected_Peaks)):
            #print(type(Selected_Peaks[i]))
            if (isinstance(Selected_Peaks[i-1], np.int64) and isinstance(Selected_Peaks[i], np.int64) and Selected_Peaks[i] != Selected_Peaks[i-1]) or (isinstance(Selected_Peaks[i-1], int) and isinstance(Selected_Peaks[i], int) and Selected_Peaks[i] != Selected_Peaks[i-1]):
                Beat = signal[Selected_Peaks[i-1]:Selected_Peaks[i]]
                #print('Beat Number: ' + str(i) + '  Beat Size: ' + str(Beat.shape[0]) + f"    Start Index : {Selected_Peaks[i-1]}    End Index : {Selected_Peaks[i]}")
                t = np.linspace(-1,1,Beat.shape[0])
                Coeff = chebyshev.chebfit(x = t,y = Beat,deg=(N-1))
                Feature = ''
                for element in Coeff:
                    Feature += ',' + str(element)
                
                file.write(Audio + f'_{str(i-1)}' + Feature + '\n')
    except:
        print("Skipping the Audio: " + Audio )


## **FEATURE EXTRACTION (BEATWISE INTERVAL)**

In [ ]:
Degrees = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
Peak = 'T'  # P,Q,R,S,T (Possible Inputs)

os.mkdir(dir + f'/Feature_Dataset/{Peak}{Peak}_Interval')

for Degree in Degrees:
    os.mkdir(dir + f'/Feature_Dataset/{Peak}{Peak}_Interval/{str(Degree)}')
    for Person in HC:
        
        file = open( dir + f'/Feature_Dataset/{Peak}{Peak}_Interval/{str(Degree)}/Feature_Extraction_{Peak}{Peak}_Interval_{Degree}_{Person[0]}.csv', 'a')

        file.write('ID')
        for k in range(1,Degree+1):
            file.write(f',CC_{k}')
        file.write('\n')

        for Audio in Person[1]:
            print("Person :" + Person[0] + '   Audio: ' + Audio)
            Feature_Extraction(Patient=Person[0], Audio = Audio, N = Degree, SR= 1000, lead = 11, Peak_type=Peak, file = file)
            
        file.close()


## **TESTING AREA**

In [5]:
SR = 1000
lead = 11
N = 5
Peak_type = 'S'


signal_array, fields = wf.rdsamp('/home/senume/Project/Research/Person-Identification-using-Chebyshev-polynomial/Dataset/patient001/s0010_re')
signal = signal_array[:,lead]

signal = ECG.ecg_clean(signal, sampling_rate = SR, method='pantompkins1985')

_,R_Peaks = ECG.ecg_peaks(signal, Sampling_rate = SR, method='pantompkins1985')
_,Peaks = ECG.ecg_delineate(signal, R_Peaks, sampling_rate = SR, show_type= 'all', method= 'cwt')

Peaks.update(R_Peaks)

# Selected_Peaks = Peaks[f"ECG_{Peak_type}_Peaks"]

# #file = open(dir + "/Feature_Dataset/" + str(N) + "/" + f"Feature_Extraction_{Patient}")

# for i in range(1, len(Selected_Peaks)):
#     if Selected_Peaks[i] != np.nan and Selected_Peaks[i-1] != np.nan:
#         Beat = signal[Selected_Peaks[i-1]:Selected_Peaks[i]]
#         t = np.linspace(-1,1,Beat.shape[0])
#         Coeff = chebyshev.chebfit(t,Beat,N)

# print(Coeff)

Peaks['']




[627,
 1185,
 1650,
 2649,
 3099,
 4122,
 4868,
 5318,
 6062,
 6803,
 7796,
 8244,
 8982,
 9964,
 10418,
 11147,
 11869,
 12852,
 13309,
 14042,
 15047,
 15783,
 16240,
 17263,
 17985,
 18714,
 19453,
 20184,
 20905,
 21358,
 22083,
 23103,
 23830,
 24559,
 25295,
 26019,
 26761,
 27225,
 27955,
 28971,
 29711,
 30460,
 31186,
 31931,
 32375,
 33415,
 34149,
 34901,
 35355,
 36390,
 37123,
 37872]